In [1]:
from os.path import join, basename, splitext
import json
from glob import glob
from tqdm import tqdm
from itertools import islice
import numpy as np

from common import tokenize
from nltk.tokenize import sent_tokenize
import logging

from IPython.display import Markdown, display, HTML
from common import get_all_docs, word_tokenizer

logger = logging.getLogger(__name__)
DATA_FOLDER = '../data/'

Using TensorFlow backend.


In [2]:
all_docs = get_all_docs(DATA_FOLDER)
val_docs = glob(join(DATA_FOLDER, 'validate/*.txt'))

with open(join(DATA_FOLDER, 'best.json'), 'r') as f:
    best = json.load(f)
    
with open(join(DATA_FOLDER, 'gold.json'), 'r') as f:
    gold = json.load(f)

In [3]:
def hprint(string, i=1):
    display(HTML('<h%s>%s</h%s>' % (i, string, i)))

def print_file(sub_name, domain, i=1, tokenize=False):    
    found = [fn for fn in domain if sub_name in fn]
    for fn in found:
        hprint(fn + '\n', i)
        with open(fn, 'r') as f:
            text = f.read()
            if tokenize:
                print(tokenize(text))
            else:
                print(text)
#         !cat {'\''+fn+'\''}

def inspect_result(qname, start_index=0, end_index=2):
    true_docs = gold[qname]

    hprint('Query\n')
    print_file(qname, val_docs, 5)
    for i, tname in enumerate(true_docs):
        hprint('True %s\n' %(i+1))
        print_file(tname, all_docs, 5)

    for i, bname in enumerate(best[qname][start_index:end_index]):
        hprint('Found %s\n' % (i+1))
        print_file(bname, all_docs, 5)    

In [4]:
val_docs_ids = set([splitext(basename(f))[0] for f in val_docs])
validation_ids = [k for k in gold.keys() if k in val_docs_ids]
print('validation set size %s' % len(validation_ids))
for qname in validation_ids:
    print(qname, set(best[qname][:20]).intersection(gold[qname]))

validation set size 130
_40_1a_2011126928A10010101RU {'2006113775A20071120RU'}
_df_5c_2010114200A10010101RU {'0097117509A19990720RU'}
_ab_68_2006139128A10010101RU {'0002093502C119971020RU'}
_1b_60_2010122997A10010101RU {'2007127531A20090127RU'}
_ef_e0_2010102239A10010101RU set()
_a5_03_2009140669A10010101RU set()
_87_7b_2010120096A10010101RU {'0002333360C120080910RU'}
_c3_b8_2009111341A10010101RU {'0002242493C120041220RU'}
_ff_96_2010138984A10010101RU {'0002168617C220010610RU'}
_be_ef_2010152978A10010101RU set()
_59_59_2009131618A10010101RU {'0002316413C120080210RU'}
_87_6d_2008148713A10010101RU set()
_0c_89_2009110686A10010101RU {'0002270304C120060220RU'}
_03_43_2008151996A10010101RU {'0000036681U120040320RU'}
_94_b1_2009122096A10010101RU set()
_e6_cc_2009147572A10010101RU set()
_18_b9_2010101690A10010101RU {'2005105373A20060810RU'}
_79_fe_2010107007A10010101RU set()
_ae_fc_2010107173A10010101RU set()
_8d_51_2010136608A10010101RU set()
_05_7a_2007148915A10010101RU set()
_82_1b_2010112

# Характеристика качества данных

| Заявка | Порочащий д-т без описания   | Хорошее совпадение назаний| Обычный человек отличит|
|------|------|------|
|   _40_1a_2011126928A10010101RU  | 1|||
|   _df_5c_2010114200A10010101RU  | 1|1||
|   _0c_89_2009110686A10010101RU  | |||

# Подход

Так как весь текст патента избыточен для выяснения конкретного новшества данного патента, то имеет смысл брать куски текста вокруг ключевых слов, указывающих на само новшество. Ниже пример таких слов.

Весь текст патента избыточен еще и потому что суть изобретения может быть применима в разных областях, например "кулинария" (подогрев плиты) и "одежда"(подогрев одежды). 

Поиск по ключевым словам:
- из названия патента
- заявлять
- заключать в том , что
- суть
- новый
- применение в качество
- новый технический результат заключаться в возможность
- новый технологичный способ получение
- преимущество
- сущность изобретение
- эффективный метод, средство, способ
- удалось
- задача , на решение который направлять настоящий изобретение
- новизна
- предмет изобретение
- описывать способ получение
- технический результат полезный модель заключаться
- полезность изобретение
    
Можно дополнить этот список наиболее близкими словами к данным, полученных на этом корпусе.
    
Имеет смысл разметить в тексте патентов такие куски, которые указывают на саму суть и обучаться на двух трех предложениях вокруг ключевых слов. Таким образом есть вероятность что мы лучше поймем отличия патентов.

Метод - CNN, которые выделяют наиболее значимые куски текста, делают саммаризацию в режиме unsupervised learning.

Простой способ индикации описания сути изобретения - векторное среднее движущегося n-gramm по тексту. Нужно пройтись так по всему корпусу для каждого предложения, получить векторные представления n-gramm(фраз-индикаторов), взять n ближайших соседий от заранее заданных фраз-индикаторов(по типу тех что приведены выше) - это и будут фразы-индикаторы. При этом можно будет придумывать новые фразы и смотреть, попадают ли они в этот кластер. Также из n-gramm можно будет выкидывать слова, которые не имеют целевой значимости, например:

"РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула".


In [5]:
import gensim

model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/old/w2v_model_300'))
wv = model.wv

In [6]:
wv.most_similar(positive=['результат'], topn=10)

[('получать', 0.666301429271698),
 ('предлагать', 0.6384239792823792),
 ('неудачн', 0.6303017735481262),
 ('заключаться', 0.6248302459716797),
 ('изобретение', 0.597603440284729),
 ('обеспечивать', 0.584223747253418),
 ('использование', 0.5819722414016724),
 ('существенный', 0.579962432384491),
 ('достигаться', 0.5796698927879333),
 ('достигаемый', 0.5791462659835815)]

In [7]:
def most_probable(wv, words, n=4, enable_logging=True):
    if len(words) <= n:
        return words

    wv.init_sims()
    
    used_words = set([word for word in words if word in wv])
    if len(used_words) != len(words):
        ignored_words = set(words) - set(used_words)
        if enable_logging:
            logger.warning("vectors for words %s are not present in the model, ignoring these words", ignored_words)
    if not used_words:
        logger.warning('not in vocabulary %s', words)
        return []
    vectors = np.vstack(wv.word_vec(word, use_norm=True) for word in used_words).astype(np.float32)
    mean = gensim.matutils.unitvec(vectors.mean(axis=0)).astype(np.float32)
    dists = np.dot(vectors, mean)
    most = sorted(zip(dists, used_words), reverse=True)[:n]
    
    return [word for sim, word in most]

In [8]:
sent = 'РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула'
words = sent.lower().split()
splited = most_probable(wv, words, n=4)

print('final n-gram: %s' % splited)

final n-gram: ['описывать', 'получение', 'формула', 'способ']


In [9]:
sent = 'технический результат полезный модель заключаться'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['заключаться', 'результат', 'технический']


In [10]:
sent = 'новый технологичный способ получение'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['способ', 'получение', 'новый']


#### Теперь придумаем новую фразу

In [11]:
sent = 'новизна данный изобретение в том что гидроусилитель статический'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'в', 'что', 'том'} are not present in the model, ignoring these words


final n-gram: ['данный', 'новизна', 'изобретение']


Здесь важно натренировать w2v так чтобы в корпусе присутствовали все слова кроме стоп слов, например, если убрать "данный", то получаем:

In [12]:
sent = 'новизна изобретение в том что гидроусилитель статический '
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'в', 'что', 'том'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'изобретение', 'статический']


А это уже хуже. Поэтому надо брать достаточное скользящее окно, например, 5 и в нем выбирать 3 наиболее вероятных терма. Также плохо брать слишком большое окно

In [13]:
sent = 'новизна изобретение в том что гидроусилитель статический с повышенный уровень отказоустойчивость'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'с', 'в', 'что', 'том'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'отказоустойчивость', 'уровень']


# Реализация

In [14]:
def iconv(sent, n=3):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(sent)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [15]:
def convolv_on_doc(doc, window=5, n=3):
    """
    doc - list object with sentences as lists of words
    window - windows size
    n - n-gram size
    """
    for sent in doc:
        step = max(n, min(window, len(sent)))
        for ngram in iconv(sent, step):
            yield ngram

**Explanation**

In [16]:
doc = [[1,2],[3,4,5],[6,7,8,9],[10,11,12,13,14],[15,16,17,18,19,20]]
for tup in convolv_on_doc(doc):
    print(tup)

(3, 4, 5)
(6, 7, 8, 9)
(10, 11, 12, 13, 14)
(15, 16, 17, 18, 19)
(16, 17, 18, 19, 20)


In [17]:
from nltk.corpus import stopwords

stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list = set(stop_list)


fn = '../data/validate/_ab_68_2006139128A10010101RU.txt'
with open(fn, 'r') as f:
    doc_text = f.read()

sents = sent_tokenize(doc_text)

doc = [tokenize(s, stop_list) for s in sents]

In [18]:
def get_convolved_ngrams(doc, window=5, n=3):
    result = []
    for ngram in convolv_on_doc(doc, window, n):
        probable_ngram = most_probable(wv, ngram, n, enable_logging=False)
        if len(probable_ngram):
            avg_ngram = np.mean(wv[probable_ngram], axis=0)
            result.append((probable_ngram, avg_ngram))
    return result

In [19]:
convolved_doc = get_convolved_ngrams(doc, window=5, n=3)

not in vocabulary ('crc', 'handbook', 'of', 'chemistry', 'and')
not in vocabulary ('handbook', 'of', 'chemistry', 'and', 'physics')
not in vocabulary ('of', 'chemistry', 'and', 'physics', 'edition')
not in vocabulary ('cf', 'ch', 'chf', 'hfc', 'ze')
not in vocabulary ('cf', 'cf', 'ch', 'chf', 'hfc')
not in vocabulary ('cf', 'ch', 'chf', 'hfc', 'mce')
not in vocabulary ('cf', 'ch', 'chc', 'ch', 'chc')
not in vocabulary ('cf', 'cf', 'ch', 'chc', 'ch')
not in vocabulary ('cf', 'ch', 'chc', 'ch', 'chc')


In [20]:
len(convolved_doc)

1192

# Пример

In [25]:
inspect_result('_2d_cb_2010113518A10010101RU', 0, 20)
# показательный пример того что наличие общих фраз еще ничего не гарантирует _fc_bc_2010110681A10010101RU

﻿способ контроль параметр груз ( вариант ) .
 изобретение относиться к складский хозяйство , и мочь быть использовать для контроль складирование , перемещение и сбыт груз , при проведение учет и инвентаризация склад .
 способ контроль параметр груз заключаться в предварительный ввод в система управление склад данные о параметр груз и обязательный условие по количество совпадение с описание параметр груз между себя и / или с данные система управление склад , где с =2…n , и разрешенный количество попытка п составление описание параметр груз в случай недостаточный количество совпадение описание , где п ≥ с -1 , а также вариант обработка исключительный ситуация .
 перед начало контроль осуществлять авторизация в система управление склад , идентифицировать груз , без доступ к данный в система управление склад составлять описание о параметр , производить сравнение составлять описание между себя и / или с данные система управление склад , определять текущий количество совпадение ст и сравнива

способ контроль подлинность и перемещение алкогольный продукция ( вариант )  . 
 изобретение относиться к область электронный информационный система .
 технический результат заключаться в повышение достоверность и сокращение срок определение подлинность продукция , а также повышение эффективность контроль за подлинный продукция и ее перемещение .
 в способ каждый единица продукция присваивать идентификационный код , который вводить в база данные сервер производитель , в качество носитель идентификационный код использовать радиочастотный метка в вид транспондер со встраивать антенный и микросхема с записать в она идентификационный код , который снабжать единица продукция на конвейер при производство , при перемещение продукция на склад или пункт реализация , осуществлять считывание идентификационный код и ввод этот данные в база данные на сервер склад или пункт реализация и одновременно в база данные сервер фискальный орган .
 при контроль сравнивать считать и запомненный код .
 4 н . п

способ контроль параметр груз ( вариант )  . 
 1 .
 способ контроль параметр груз , включать предварительный ввод данные о параметр груз в система управление склад , идентификация груз , составление описание параметр груз , сравнение описание параметр груз с данные система управление склад и принятие решение о дальнейший действие с груз по результат сравнение , отличать тем , что в система управление склад дополнительно вводить обязательный условие по количество совпадение с описание параметр груз между себя и/или с данные система управление склад , где с=2…n , и разрешенный количество попытка п составление описание параметр груз в случай недостаточный количество совпадение описание , где п≥с-1 , а также вариант обработка исключительный ситуация , перед начало контроль осуществлять авторизация в система управление склад , описание параметр груз составлять без доступ к данный в система управление склад , производить сравнение описание параметр груз между себя и/или с данные система упра

система для организация работа сотрудник в отдел  . 
 1 .
 система управление выполнение задание в розничный торговля , содержать : a .
 магазин , в который иметься работник , находить при исполнение служебный обязанность , и b .
 сеть связь для осуществление работник магазин электронный связь в магазин , причем c .
 по меньший мера один работник магазин иметь устройство связь менеджер , d .
 по меньший мера один работник магазин иметь устройство связь рядовой сотрудник магазин , а работник магазин связываться друг с друг по сеть связь для тот , чтобы давать и выполнять перечень задание , который необходимо выполнять в магазин .
 2 .
 система по п.1 , в который в число работник магазин входить по меньший мера один менеджер и по меньший мера один рядовой сотрудник магазин , находить в подчинение у указанный по меньший мера один менеджер .
 3 .
 система по п.2 , в который указанный по меньший мера один менеджер формировать перечень задание на устройство связь менеджер , а устройство связ

автоматизированный система контроль за аттестация персонал на производство ( вариант )  . 
 1 .
 автоматизированный система контроль за аттестация персонал на производство , содержать индивидуальный носитель информация , устройство считывание информация с указанный носитель , блок обработка информация , средство выполнение команда , поступать из блок обработка информация , и блок хранение информация , отличать тем , что блок хранение информация дополнительно содержать модуль хранение информация о срок и результат аттестация по меньший мера один лицо , индивидуальный носитель информация содержать сведение о срок и результат аттестация конкретный лицо , блок обработка информация выполнять с возможность анализ информация , содержать на индивидуальный носитель и ее сопоставление с информация , находить в модуль хранение , средство выполнение команда , поступать из блок обработка информация , выполнять в вид запорный механизм , предоставлять или запрещать проход лицо на контролируемый терри

...

автоматизированный система информационный поддержка система качество при производство конструкционный сложный изделие  . 
 автоматизированный система информационный поддержка система качество при производство конструкционный сложный изделие , содержать автоматизированный рабочий место оператор с первое по четвертый и блок обработка данные , отличать тем , что вводить блок управление карта процесс , первый вход который соединенный с выход первое автоматизированный рабочий место , блок формирование регламент выполнение и контроль процесс , первый вход который соединенный с первое выход блок управление карта процесс , блок формирование карта процесс на основание регламент , выход который соединенный со второе вход блок управление карта процесс , блок обработка нормативно-распорядительных документ , вход который соединенный с выход второе автоматизированный рабочий место оператор , а выход - соединенный со второе вход блок формирование регламент выполнение и контроль процесс , блок согласо

автоматизированный система управление единый дежурный служба  . 
 1 .
 автоматизированный система управление единый дежурный служба , содержать : подсистема взаимодействие с пользователь , который обеспечивать сбор , обработка , хранение и ведение информация об обращение в территориальный управление отдельный гражданин и организация о наступать чрезвычайный ситуация , связанный с подсистема управление доступ к данный , обеспечивать разграничение правый доступ пользователь к данный в зависимость от выдавать разрешение , контроль за ход устранение аварийный и чрезвычайный ситуация , выполнять с возможность управление данные ; и подсистема информационный взаимодействие , обеспечивать информационный взаимодействие между территориальный управление и отдел единый дежурный служба , между территориальный управление и подведомственный организация и координация деятельность дежурный и диспетчерский служба по ликвидация последствие происшествие и чрезвычайный ситуация посредством канал связь ; по

устройство контроль рабочий группа в система для контроль личной присутствие  . 
 устройство контроль рабочий группа , включать блок считывание , блок передача , отличать тем , что в о состав дополнительно вводить блок сравнение идентификационный информация , устройство контроль рабочий группа включать в линия информационный связь между устройство регистрация прибытие персонал и сервер безопасность корпоративный локальный вычислительный сеть , при этот линия связь выполнять однонаправленный .  .   .   .


способ автоматизация рабочий место специалист по охрана труд и устройство для о осуществление  . 
 изобретение относиться к вычислительный техника .
 о использование при автоматизация рабочий место специалист по охрана труд позволять повышать эффективность принимать этот специалист профессиональный решение .
 способ реализоваться в устройство , содержать связанный интерфейс конечный пользователь блок формирование база данные , блок обработка и блок принятие решение , а также блок средство передача , соединенный с блок формирование база данные , и машиноориентированный информационный носитель .
 технический результат достигаться благодаря тот , что информационный носитель иметь определенный структура с настраивать содержание в соответствие с потребность конкретный производство , согласованный с действующий утверждать отчетность , а блок обработка и блок принятие решение снабжать программный средство для обеспечение автоматизированный учет сведение и знание о состояние безопасность и усл

автоматизированный система сопровождение , контроль и управление жизненный цикл товар и способ ее функционирование  . 
 1 .
 автоматизированный система сопровождение , контроль и управление жизненный цикл товар , содержать базовый аналитический компьютерный сервер-центр с выбор о месторасположение в любой материальный среда с возможность управление через любой среда и совокупность взаимосвязанный между они и себя коммуникационный линия локальный сервер , соединенный с база управление оптимизация этап жизненный цикл каждый вид товар ( далее эжцквт ) , с расположение в она автоматизированный средство централизованный управление и контроль объем и параметр создавать , прошедший по все этап эжцквт и доставлять на конечный пункт эжцквт товар в полный и/или частичный объем , в натуральный и/или перерабатывать вид , и/или в вид полуфабрикат , и/или группа , и/или каждый экземпляр товар , который снабжать электронный бирка и/или контрольный аналитический узел , с выбирать вид радиочастотный со